In [90]:
import pandas as pd
import numpy as np


In [91]:
df = pd.read_csv("diabetes.csv")

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [92]:
X = df.drop("Outcome",axis=1)
Y = df[["Outcome"]]

In [93]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = scaler.fit_transform(X)

In [94]:
from sklearn.model_selection import train_test_split

X_train , X_test ,Y_train, Y_test = train_test_split(X,Y,test_size=0.3,random_state=1)

In [131]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense ,Dropout

In [96]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim = 8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics=['accuracy'])

In [97]:
model.fit(X_train,Y_train,batch_size=32,epochs=100,validation_data=(X_test,Y_test))

Epoch 1/100


17/17 [==============================] - 1s 14ms/step - loss: 0.7154 - accuracy: 0.5177 - val_loss: 0.7198 - val_accuracy: 0.5195
Epoch 2/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6757 - accuracy: 0.5829 - val_loss: 0.6815 - val_accuracy: 0.5801
Epoch 3/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6451 - accuracy: 0.6369 - val_loss: 0.6502 - val_accuracy: 0.6234
Epoch 4/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6201 - accuracy: 0.6462 - val_loss: 0.6245 - val_accuracy: 0.6623
Epoch 5/100
17/17 [==============================] - 0s 5ms/step - loss: 0.5986 - accuracy: 0.6704 - val_loss: 0.6026 - val_accuracy: 0.6753
Epoch 6/100
17/17 [==============================] - 0s 5ms/step - loss: 0.5806 - accuracy: 0.6834 - val_loss: 0.5851 - val_accuracy: 0.6883
Epoch 7/100
17/17 [==============================] - 0s 5ms/step - loss: 0.5654 - accuracy: 0.6853 - val_loss: 0.5692 - val_accuracy: 0.6926
Epoch 8/100
17/17 [=====

# steps for tuning the layer
     1) how to select appropiate optimizer
     2) Number of nodes in a layer
     3) how to select the number of hidden layers in a model
     4) all in model( with all the necesary features)

In [98]:
import keras_tuner as kt

In [99]:
def build_model(hp):

    model = Sequential()

    model.add(Dense(32,activation='relu',input_dim = 8))
    model.add(Dense(1,activation ='sigmoid'))

    optimizer = hp.Choice('optimizer',values =['adam','sgd','rmsprop','adadelta'])

    model.compile(optimizer=optimizer,loss = 'binary_crossentropy',metrics=['accuracy'])

    return model

In [100]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [101]:
tuner.search(X_train,Y_train ,epochs =5,validation_data =(X_test,Y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.4545454680919647

Best val_accuracy So Far: 0.7835497856140137
Total elapsed time: 00h 00m 10s
INFO:tensorflow:Oracle triggered exit


In [102]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [103]:
model = tuner.get_best_models(num_models=1)[0]

In [104]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [105]:
model.fit(X_train,Y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,Y_test))

Epoch 7/100
17/17 [==============================] - 1s 15ms/step - loss: 0.5377 - accuracy: 0.7449 - val_loss: 0.5169 - val_accuracy: 0.7835
Epoch 8/100
17/17 [==============================] - 0s 5ms/step - loss: 0.5229 - accuracy: 0.7561 - val_loss: 0.5054 - val_accuracy: 0.7835
Epoch 9/100
17/17 [==============================] - 0s 5ms/step - loss: 0.5127 - accuracy: 0.7616 - val_loss: 0.4962 - val_accuracy: 0.7879
Epoch 10/100
17/17 [==============================] - 0s 5ms/step - loss: 0.5043 - accuracy: 0.7616 - val_loss: 0.4871 - val_accuracy: 0.7879
Epoch 11/100
17/17 [==============================] - 0s 5ms/step - loss: 0.4977 - accuracy: 0.7598 - val_loss: 0.4805 - val_accuracy: 0.7922
Epoch 12/100
17/17 [==============================] - 0s 5ms/step - loss: 0.4923 - accuracy: 0.7616 - val_loss: 0.4755 - val_accuracy: 0.7922
Epoch 13/100
17/17 [==============================] - 0s 6ms/step - loss: 0.4878 - accuracy: 0.7654 - val_loss: 0.4710 - val_accuracy: 0.8009
Epoch 14

# now we are figuring out number of neurons in a particular layer in the model

In [106]:
def build_model(hp):
    model = Sequential()

    units = hp.Int('units',min_value =8,max_value =128,step =8)
    
    model.add(Dense(units=units,activation='relu',input_dim = 8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics=['accuracy'])

    return model

In [107]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy', max_trials=5,directory ='mydir',project_name = 'Dipayan')

In [108]:
tuner.search(X_train,Y_train,epochs =5,validation_data =(X_test,Y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8138527870178223
Total elapsed time: 00h 00m 14s
INFO:tensorflow:Oracle triggered exit


In [109]:
tuner.get_best_hyperparameters()[0].values

{'units': 96}

In [110]:
model = tuner.get_best_models(num_models=1)[0]

In [111]:
model.fit(X_train,Y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
17/17 [==============================] - 1s 2ms/step - loss: 0.5499 - accuracy: 0.7467
Epoch 8/100
17/17 [==============================] - 0s 3ms/step - loss: 0.5234 - accuracy: 0.7561
Epoch 9/100
17/17 [==============================] - 0s 2ms/step - loss: 0.5054 - accuracy: 0.7579
Epoch 10/100
17/17 [==============================] - 0s 3ms/step - loss: 0.4916 - accuracy: 0.7561
Epoch 11/100
17/17 [==============================] - 0s 3ms/step - loss: 0.4823 - accuracy: 0.7598
Epoch 12/100
17/17 [==============================] - 0s 3ms/step - loss: 0.4758 - accuracy: 0.7728
Epoch 13/100
17/17 [==============================] - 0s 3ms/step - loss: 0.4701 - accuracy: 0.7728
Epoch 14/100
17/17 [==============================] - 0s 2ms/step - loss: 0.4665 - accuracy: 0.7784
Epoch 15/100
17/17 [==============================] - 0s 2ms/step - loss: 0.4629 - accuracy: 0.7803
Epoch 16/100
17/17 [==============================] - 0s 2ms/step - loss: 0.4606 - accuracy: 0.7765
Epo

# Select the number of Layers in a model

In [112]:
def build_model(hp):

    model = Sequential()

    model.add(Dense(96,activation ='relu',input_dim =8))

    for i in range(hp.Int('num_layers',min_value = 1,max_value = 10)):

        model.add(Dense(96,activation ='relu'))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics=['accuracy'])

    return model

In [113]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory ='layer_dir',project_name = 'num_of_layers')

In [114]:
tuner.search(X_train,Y_train,epochs =5,validation_data = (X_test,Y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.8095238208770752

Best val_accuracy So Far: 0.8138527870178223
Total elapsed time: 00h 00m 22s
INFO:tensorflow:Oracle triggered exit


In [115]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [116]:
model = tuner.get_best_models(num_models=1)[0]


In [117]:
model.fit(X_train,Y_train,epochs=100,initial_epoch=6,validation_data=(X_test,Y_test))

Epoch 7/100
17/17 [==============================] - 2s 17ms/step - loss: 0.4646 - accuracy: 0.7654 - val_loss: 0.4491 - val_accuracy: 0.7922
Epoch 8/100
17/17 [==============================] - 0s 7ms/step - loss: 0.4486 - accuracy: 0.7952 - val_loss: 0.4394 - val_accuracy: 0.8139
Epoch 9/100
17/17 [==============================] - 0s 9ms/step - loss: 0.4372 - accuracy: 0.7858 - val_loss: 0.4577 - val_accuracy: 0.7835
Epoch 10/100
17/17 [==============================] - 0s 7ms/step - loss: 0.4301 - accuracy: 0.7840 - val_loss: 0.4372 - val_accuracy: 0.8009
Epoch 11/100
17/17 [==============================] - 0s 7ms/step - loss: 0.4208 - accuracy: 0.7914 - val_loss: 0.4376 - val_accuracy: 0.8009
Epoch 12/100
17/17 [==============================] - 0s 7ms/step - loss: 0.4173 - accuracy: 0.8119 - val_loss: 0.4380 - val_accuracy: 0.8009
Epoch 13/100
17/17 [==============================] - 0s 6ms/step - loss: 0.3979 - accuracy: 0.8063 - val_loss: 0.4367 - val_accuracy: 0.7922
Epoch 14

# last bit to determine all the things at once


In [132]:
def build_model(hp):

    model = Sequential()

    counter = 0
    

    for i in range (hp.Int("num_layers",min_value =1 ,max_value = 10)):

        if counter == 0:

            model.add(
                Dense(
                hp.Int('units' + str(i),min_value =8,max_value = 128,step =8),
                      activation = hp.Choice('activation'+str(i),values =['relu','tanh','sigmoid']),
                      input_dim = 8
            )
        )
            model.add(Dropout(hp.Choice('Dropout'+ str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
            
        else:
            model.add(
                Dense(
                hp.Int('units' + str(i),min_value = 8,max_value = 128,step =8),
                activation=hp.Choice('activation_'+str(i) , values=['relu', 'tanh','sigmoid'])
            )
        )
            model.add(Dropout(hp.Choice('Dropout'+ str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter+=1


    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer',values = ['rmsprop','adam','sgd','nadam','adadelta']),
                  loss ='binary_crossentropy',
                  metrics=['accuracy'])
    return model





    


In [133]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy', max_trials=5,
                        directory ='final_all',
                        project_name = 'final')

In [134]:
tuner.search(X_train,Y_train,epochs = 5,validation_data =(X_test,Y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.36796537041664124

Best val_accuracy So Far: 0.6406926512718201
Total elapsed time: 00h 00m 20s
INFO:tensorflow:Oracle triggered exit


In [135]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8,
 'units0': 128,
 'activation0': 'tanh',
 'Dropout0': 0.9,
 'optimizer': 'adam',
 'units1': 8,
 'activation_1': 'relu',
 'Dropout1': 0.1,
 'units2': 8,
 'activation_2': 'relu',
 'Dropout2': 0.1,
 'units3': 8,
 'activation_3': 'relu',
 'Dropout3': 0.1,
 'units4': 8,
 'activation_4': 'relu',
 'Dropout4': 0.1,
 'units5': 8,
 'activation_5': 'relu',
 'Dropout5': 0.1,
 'units6': 8,
 'activation_6': 'relu',
 'Dropout6': 0.1,
 'units7': 8,
 'activation_7': 'relu',
 'Dropout7': 0.1}

In [136]:
model = tuner.get_best_models(num_models=1)[0]

In [137]:
model.fit(X_train,Y_train,batch_size=32,epochs=200,initial_epoch=6,validation_data=(X_test,Y_test))

Epoch 7/200
17/17 [==============================] - 3s 22ms/step - loss: 0.6834 - accuracy: 0.6015 - val_loss: 0.6785 - val_accuracy: 0.6320
Epoch 8/200
17/17 [==============================] - 0s 8ms/step - loss: 0.6694 - accuracy: 0.6480 - val_loss: 0.6715 - val_accuracy: 0.6320
Epoch 9/200
17/17 [==============================] - 0s 7ms/step - loss: 0.6629 - accuracy: 0.6574 - val_loss: 0.6639 - val_accuracy: 0.6320
Epoch 10/200
17/17 [==============================] - 0s 7ms/step - loss: 0.6534 - accuracy: 0.6574 - val_loss: 0.6579 - val_accuracy: 0.6320
Epoch 11/200
17/17 [==============================] - 0s 7ms/step - loss: 0.6486 - accuracy: 0.6704 - val_loss: 0.6542 - val_accuracy: 0.6320
Epoch 12/200
17/17 [==============================] - 0s 8ms/step - loss: 0.6535 - accuracy: 0.6592 - val_loss: 0.6495 - val_accuracy: 0.6320
Epoch 13/200
17/17 [==============================] - 0s 7ms/step - loss: 0.6406 - accuracy: 0.6611 - val_loss: 0.6456 - val_accuracy: 0.6320
Epoch 14